In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "20"
from collections import defaultdict
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
folder = "../data/cna/genosurf_download/files/"

In [3]:
files = os.listdir(folder)
files = filter(lambda x:x.endswith(".bed"), files)
files = list(files)
s = set()
d = defaultdict(int)

d_new = defaultdict(int)



check = False

max_ = 0
min_ = 0
min_pos = 10000000
max_neg = -10000000

counts = defaultdict(int)

for file_name in tqdm(files):
    file_name = folder + file_name
    if check:
        pre_chr = None
    with open(file_name) as file:
#         print(file_name)
        file = map(lambda x:x.strip().split("\t"), file)
        file = map(lambda x:(x[0],int(x[1]), int(x[2]),x[3],int(float(x[4])),float(x[5]),x[6]), file)
        file = filter(lambda x:x[6] == 'Y' , file)
        
        if check:
            file = sorted(file)
                
        for (chrom, start,stop,strand,count,value,nocnv) in file:
#             print(file_name,(chrom, start,stop,strand,count,value,nocnv))
            if check: # check is there any overlap !!!!
                if pre_chr != chrom:
                    pre_chr = chrom
                else:
                    if start < pre_stop:
                        print(pre_stop,"********************************************************************************")
                        print(chrom, start,stop,strand,count,value,nocnv)

                pre_stop = stop
            
            s.add((chrom,start))
            s.add((chrom,stop))
            d[(chrom, start,stop)] = d[(chrom, start,stop)] + 1
            
            d_new[(chrom,start)] = d_new[(chrom,start)] + 1
            d_new[(chrom,stop)] = d_new[(chrom,stop)] - 1
            
            max_ = max(max_, value)

            min_ = min(min_, value)
            if value > 0:
                min_pos = min(min_pos, value )
            if value < 0:
                max_neg = max(max_neg, value)
                
            if value == 0:
                counts['value == 0'] = counts['value == 0']  + 1
            
            if value < 0:
                counts['value < 0'] = counts['value < 0']  + 1
                
            if value > 0:
                counts['value > 0'] = counts['value > 0']  + 1
            
            if value > 1:
                counts['value > 1'] = counts['value > 1']  + 1
            if value < -1:
                counts['value < -1'] = counts['value < -1']  + 1
                
            if value > 2:
                counts['value > 2'] = counts['value > 2']  + 1
            if value < -2:
                counts['value < -2'] = counts['value < -2']  + 1
            
            counts['all'] = counts['all']  + 1
            
        
#         break
        


100%|██████████| 22632/22632 [03:23<00:00, 111.09it/s]


In [92]:
d_new

defaultdict(int,
            {('chr15', 23687685): 22631,
             ('chr15', 41937460): -4,
             ('chr15', 41950155): 4,
             ('chr15', 41991076): 0,
             ('chr19', 10177629): -4,
             ('chr19', 10656442): -3,
             ('chr19', 10656804): 4,
             ('chr19', 34599808): 12,
             ('chr19', 34600936): 0,
             ('chr19', 34661697): -1,
             ('chr19', 34667849): 2,
             ('chr19', 35075455): -4,
             ('chr19', 35084293): 8,
             ('chr19', 35154447): -8,
             ('chr19', 35156245): 4,
             ('chr19', 52270087): -38,
             ('chr19', 52624873): 35,
             ('chr19', 52653973): 0,
             ('chr15', 41992272): -9,
             ('chr15', 101884307): -22630,
             ('chr19', 52657023): 2,
             ('chr19', 58878226): -22632,
             ('chr16', 653459): 22552,
             ('chr16', 901174): -2,
             ('chr20', 455764): 22552,
             ('chr20', 207554

In [4]:
np.unique(list(map(lambda x : x[0], d_new.keys())))

array(['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr20', 'chr21',
       'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9',
       'chrX'], dtype='<U5')

In [5]:
# in order to use the positions, it is enough sets in the previous cell!!!!

chr_positions =sorted(map(lambda x : x[1], filter(lambda x : x[0] == 'chrX', d_new.keys())))
df = pd.DataFrame(index = chr_positions)
indexes = df.index
print('len(chr11_positions)',len(chr_positions))
all_series = []
for file_name in tqdm(files):
    new_series = pd.Series(np.nan, index=indexes, name=file_name)

    file_name = folder + file_name
    if check:
        pre_chr = None
    with open(file_name) as file:
#         print(file_name)
        file = map(lambda x:x.strip().split("\t"), file)
        file = map(lambda x:(x[0],int(x[1]), int(x[2]),x[3],int(float(x[4])),float(x[5]),x[6]), file)
        file = filter(lambda x:x[6] == 'Y' , file)

        file = sorted(file)

        for (chrom, start,stop,strand,count,value,nocnv) in file:
            if chrom != 'chrX':
                continue
            new_series[(start<=indexes) & (indexes<=stop)] = value
    all_series.append(new_series)


  0%|          | 12/22632 [00:00<03:17, 114.74it/s]

len(chr11_positions) 52848


100%|██████████| 22632/22632 [05:01<00:00, 75.14it/s] 


In [7]:
all_series = pd.concat(all_series, axis=1).T

In [8]:
print(all_series.shape)
all_series.head()

(22632, 52848)


,3157107,3157273,3163494,3165068,3165124,3166148,3167032,3167102,3169781,3170664,...,154883512,154883634,154884030,154884069,154884088,154895152,154895182,154895283,154896266,154905589
TCGA-AA-3693-01A-01D-0903-01.bed,0.0072,0.0072,0.0072,0.0072,0.0072,0.0072,0.0072,0.0072,0.0072,0.0072,...,-0.0004,-0.0004,-0.0004,-0.0004,-0.0004,-0.0004,-0.0004,-0.0004,-0.0004,-0.0004
TCGA-4P-AA8J-01A-11D-A390-01.bed,0.0192,0.0192,0.0192,0.0192,0.0192,0.0192,0.0192,0.0192,0.0192,0.0192,...,0.0068,0.0068,0.0068,0.0068,0.0068,0.0068,0.0068,0.0068,0.0068,0.0068
TCGA-24-0979-01A-01D-0428-01.bed,0.1520,0.1520,0.1520,0.1520,0.1520,0.1520,0.1520,0.1520,0.1520,0.1520,...,0.6317,0.6317,0.6317,0.6317,0.6317,0.6317,0.6317,0.6317,0.6317,0.6317
TCGA-4W-AA9T-01A-11D-A390-01.bed,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,...,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017
TCGA-5P-A9KC-01A-11D-A42I-01.bed,0.0047,0.0047,0.0047,0.0047,0.0047,0.0047,0.0047,0.0047,0.0047,0.0047,...,0.0222,0.0222,0.0222,0.0222,0.0222,0.0222,0.0222,0.0222,0.0222,0.0222


In [11]:
all_series.fillna(0, inplace=True)

In [8]:
all_series.columns[:-1]

Int64Index([  3218610,   3220101,   3220160,   3221051,   3221621,   3222113,
              3222334,   3222470,   3222627,   3224658,
            ...
            247792688, 247793202, 247797469, 247798522, 247800377, 247806068,
            247806930, 247807327, 247810288, 247812431],
           dtype='int64', length=106959)

In [9]:
all_series.columns[:-1]

Int64Index([  3218610,   3220101,   3220160,   3221051,   3221621,   3222113,
              3222334,   3222470,   3222627,   3224658,
            ...
            247792688, 247793202, 247797469, 247798522, 247800377, 247806068,
            247806930, 247807327, 247810288, 247812431],
           dtype='int64', length=106959)

In [12]:
corrs = []
has_nas = []
from scipy.stats import pearsonr
for x,y in tqdm(list(zip(all_series.columns[:-1],all_series.columns[1:]))):
    # in order understand the if the corrs change because of the nulls    
    has_na =  (sum(all_series[x].isna()) + sum(all_series[y].isna()) > 0)
    has_nas.append(has_na) 

    corr, z = pearsonr(all_series[x].fillna(0), all_series[y].fillna(0))
    #  print(x,y,corr, z)

    corrs.append(corr)


100%|██████████| 52847/52847 [06:37<00:00, 132.86it/s]


In [13]:
corrs[:10]

[1.0,
 0.9981826609331554,
 0.9998737381997217,
 1.0,
 0.9984791584968056,
 0.9994646039609068,
 0.9989558595339938,
 0.9987992490153113,
 0.9999351566475025,
 0.9994701132950817]

In [14]:
df = pd.DataFrame([corrs,has_nas]).T

In [15]:
df.shape

(52847, 2)

In [16]:
df.head(5)

,0,1
0,1,False
1,0.998183,False
2,0.999874,False
3,1,False
4,0.998479,False


In [17]:
corrs2 = df[df[1] == False][0]
corrs3 = df[df[1] == True][0]

In [18]:
print(corrs2.head(5))
corrs3.head(5)

0           1
1    0.998183
2    0.999874
3           1
4    0.998479
Name: 0, dtype: object


Series([], Name: 0, dtype: object)

In [19]:
from statistics import mean,median
mean(corrs),median(corrs),max(corrs),min(corrs)

(0.9830731075551162, 0.9980168258878328, 1.0, 0.39488908915125764)

In [20]:
mean(corrs2),median(corrs2),max(corrs2),min(corrs2)

(0.9830731075551162, 0.9980168258878328, 1.0, 0.39488908915125764)

In [21]:
from statistics import mean,median
mean(corrs3),median(corrs3),max(corrs3),min(corrs3)

StatisticsError: mean requires at least one data point

In [35]:
th = 0.7
print('all', len(df[(df[0] < th) ]) )
print('False', len(df[(df[0] < th) & (df[1] == False) ]) )
print('True', len(df[(df[0] < th) & (df[1] == True) ]) )

all 221
False 221
True 0


In [21]:
df[(df[0] < th)]

,0,1
4371,0.790184,True
4373,0.788609,False
9615,0.79625,True
9617,0.784862,False
12627,0.526927,False
12629,0.555799,True
18074,0.79127,False
18076,0.786956,False
18633,0.779171,False
18635,0.776697,False


In [24]:
df.loc[106958]

0    0.988294
1        True
Name: 106958, dtype: object

In [28]:
all_series.columns[106807]

247567932

In [25]:
df.tail()

,0,1
106954,0.998919,False
106955,0.994638,False
106956,0.991124,False
106957,0.99619,False
106958,0.988294,True


In [40]:
all_series.columns

Int64Index([  3218610,   3220101,   3220160,   3221051,   3221621,   3222113,
              3222334,   3222470,   3222627,   3224658,
            ...
            247793202, 247797469, 247798522, 247800377, 247806068, 247806930,
            247807327, 247810288, 247812431, 247813706],
           dtype='int64', length=106960)

In [35]:
init_idx = 0
new_df = pd.DataFrame()
for col_idx in tqdm(df[(df[0] < th)].index):
    new_df["chr1_"+str(all_series.columns[init_idx])+":"+str(all_series.columns[col_idx])] = all_series.iloc[:,init_idx:col_idx].mean(axis=1)
    init_idx = col_idx+1
new_df["chr1_"+str(all_series.columns[init_idx])+":"+str(all_series.columns[-1])] = all_series.iloc[:,init_idx:].mean(axis=1)
new_df

100%|██████████| 24/24 [00:29<00:00,  1.21s/it]


,chr1_3218610:12354307,chr1_12355487:12355503,chr1_12357088:23957783,chr1_23958921:23959613,chr1_23960868:31621427,chr1_31623667:31625346,chr1_31625389:43476957,chr1_43477796:43478606,chr1_43486624:44635476,chr1_44636481:44637390,...,chr1_88634064:120527361,chr1_149879545:211166348,chr1_211168634:211168993,chr1_211184068:216657848,chr1_216666358:230603822,chr1_230606103:230606178,chr1_230614110:238855281,chr1_238855638:247566875,chr1_247567923:247567932,chr1_247568051:247813706
TCGA-AA-3693-01A-01D-0903-01.bed,-0.006600,-0.0066,-0.006600,-0.0066,-0.006600,-0.0066,-0.006600,-0.0066,-0.006600,-0.0066,...,-0.006600,-0.006600,-0.0066,-0.006600,-0.006600,-0.0066,-0.006600,-0.005274,0.0057,0.0057
TCGA-4P-AA8J-01A-11D-A390-01.bed,-0.040900,-0.0409,-0.040900,-0.0409,-0.040900,-0.0409,-0.040900,-0.0409,-0.040900,-0.0409,...,-0.119932,0.004297,0.0355,0.035500,0.035500,0.0355,0.035500,0.020150,0.0221,0.0221
TCGA-24-0979-01A-01D-0428-01.bed,0.221400,0.2214,0.221400,0.2214,0.221400,0.2214,0.522517,0.5523,0.552300,-1.5957,...,-0.105465,-0.137501,-0.3559,-0.355900,0.152737,0.1887,-0.069629,0.193195,0.2419,0.2419
TCGA-4W-AA9T-01A-11D-A390-01.bed,0.005300,0.0053,0.005300,0.0053,0.005300,0.0053,0.005300,0.0053,0.005300,0.0053,...,0.031000,0.028092,0.0272,0.027200,0.027200,0.0272,0.027200,0.027200,0.0272,0.0272
TCGA-5P-A9KC-01A-11D-A42I-01.bed,-0.021400,-0.0214,-0.021400,-0.0214,-0.021400,-0.0214,-0.021400,-0.0214,-0.021400,-0.0214,...,-0.021400,-0.027850,-0.0332,-0.033200,-0.033200,-0.0332,-0.033200,-0.033200,-0.0332,-0.0332
TCGA-19-0955-01A-02D-0517-01.bed,-0.027700,-0.0277,-0.027700,-0.0277,-0.027700,-0.0277,-0.027700,-0.0277,-0.027700,-0.0277,...,-0.022491,-0.016100,-0.0161,-0.218337,-0.018200,-0.0182,-0.018200,-0.018200,-0.0182,-0.0182
TCGA-FS-A4FD-10B-01D-A25P-01.bed,0.000800,0.0008,0.000800,0.0008,0.000800,0.0008,0.000800,0.0008,0.000800,0.0008,...,0.000800,0.000800,0.0008,0.000800,0.000800,0.0008,0.000800,0.000800,0.0008,0.0008
TCGA-ZF-A9R5-10A-01D-A42G-01.bed,-0.006600,-0.0066,-0.001593,0.0011,0.001100,0.0011,0.001100,0.0011,0.001100,0.0011,...,0.001100,0.001100,0.0011,0.001100,0.001100,0.0011,0.001100,0.001100,0.0011,0.0011
TCGA-D3-A51J-06A-11D-A25P-01.bed,0.040800,0.0408,0.040800,0.0408,0.040800,0.0408,0.040800,0.0408,0.040800,0.0408,...,0.040800,0.040800,0.0408,0.040800,0.040800,0.0408,0.040800,0.028924,0.0469,0.0469
TCGA-KN-8433-01A-11D-2308-01.bed,-0.236300,-0.2363,-0.236300,-0.2363,-0.236300,-0.2363,-0.231442,-0.1983,-0.198300,-0.1983,...,-0.198300,-0.198300,-0.1983,-0.198300,-0.196851,-0.2086,-0.208600,-0.208600,-0.2086,-0.2086


In [156]:
df.index

RangeIndex(start=0, stop=106959, step=1)

In [151]:
new_df

,chr2_0:4371,chr2_4372:4373,chr2_4374:9615,chr2_9616:9617,chr2_9618:12627,chr2_12628:12629,chr2_12630:18074,chr2_18075:18076,chr2_18077:18633,chr2_18634:18635,...,chr2_39431:40817,chr2_40818:56357,chr2_56358:88035,chr2_88036:88038,chr2_88039:90896,chr2_90897:97734,chr2_97735:97736,chr2_97737:102561,chr2_102562:106805,chr2_106806:106807
TCGA-AA-3693-01A-01D-0903-01.bed,-0.006600,-0.0066,-0.006600,-0.0066,-0.006600,-0.0066,-0.006600,-0.0066,-0.006600,-0.0066,...,-0.006600,-0.006600,-0.006600,-0.0066,-0.006600,-0.006600,-0.0066,-0.006600,-0.005274,0.0057
TCGA-4P-AA8J-01A-11D-A390-01.bed,-0.040900,-0.0409,-0.040900,-0.0409,-0.040900,-0.0409,-0.040900,-0.0409,-0.040900,-0.0409,...,-0.054200,-0.119932,0.004297,0.0355,0.035500,0.035500,0.0355,0.035500,0.020150,0.0221
TCGA-24-0979-01A-01D-0428-01.bed,0.221400,0.2214,0.221400,0.2214,0.221400,0.2214,0.522517,0.5523,0.552300,-1.5957,...,-0.400200,-0.105465,-0.137501,-0.3559,-0.355900,0.152737,0.1887,-0.069629,0.193195,0.2419
TCGA-4W-AA9T-01A-11D-A390-01.bed,0.005300,0.0053,0.005300,0.0053,0.005300,0.0053,0.005300,0.0053,0.005300,0.0053,...,0.031000,0.031000,0.028092,0.0272,0.027200,0.027200,0.0272,0.027200,0.027200,0.0272
TCGA-5P-A9KC-01A-11D-A42I-01.bed,-0.021400,-0.0214,-0.021400,-0.0214,-0.021400,-0.0214,-0.021400,-0.0214,-0.021400,-0.0214,...,-0.021400,-0.021400,-0.027850,-0.0332,-0.033200,-0.033200,-0.0332,-0.033200,-0.033200,-0.0332
TCGA-19-0955-01A-02D-0517-01.bed,-0.027700,-0.0277,-0.027700,-0.0277,-0.027700,-0.0277,-0.027700,-0.0277,-0.027700,-0.0277,...,-0.027700,-0.022491,-0.016100,-0.0161,-0.218337,-0.018200,-0.0182,-0.018200,-0.018200,-0.0182
TCGA-FS-A4FD-10B-01D-A25P-01.bed,0.000800,0.0008,0.000800,0.0008,0.000800,0.0008,0.000800,0.0008,0.000800,0.0008,...,0.000800,0.000800,0.000800,0.0008,0.000800,0.000800,0.0008,0.000800,0.000800,0.0008
TCGA-ZF-A9R5-10A-01D-A42G-01.bed,-0.006600,-0.0066,-0.001593,0.0011,0.001100,0.0011,0.001100,0.0011,0.001100,0.0011,...,0.001100,0.001100,0.001100,0.0011,0.001100,0.001100,0.0011,0.001100,0.001100,0.0011
TCGA-D3-A51J-06A-11D-A25P-01.bed,0.040800,0.0408,0.040800,0.0408,0.040800,0.0408,0.040800,0.0408,0.040800,0.0408,...,0.040800,0.040800,0.040800,0.0408,0.040800,0.040800,0.0408,0.040800,0.028924,0.0469
TCGA-KN-8433-01A-11D-2308-01.bed,-0.236300,-0.2363,-0.236300,-0.2363,-0.236300,-0.2363,-0.231442,-0.1983,-0.198300,-0.1983,...,-0.198300,-0.198300,-0.198300,-0.1983,-0.198300,-0.196851,-0.2086,-0.208600,-0.208600,-0.2086


In [35]:
all_series.iloc[:,0:df[(df[0] < th)].index[0]]

,3218610,3220101,3220160,3221051,3221621,3222113,3222334,3222470,3222627,3224658,...,12329295,12339360,12339619,12340093,12343086,12344412,12349656,12349856,12350434,12354307
TCGA-AA-3693-01A-01D-0903-01.bed,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,...,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066
TCGA-4P-AA8J-01A-11D-A390-01.bed,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,...,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409
TCGA-24-0979-01A-01D-0428-01.bed,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,...,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214
TCGA-4W-AA9T-01A-11D-A390-01.bed,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,...,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053
TCGA-5P-A9KC-01A-11D-A42I-01.bed,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,...,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214
TCGA-19-0955-01A-02D-0517-01.bed,-0.0277,-0.0277,-0.0277,-0.0277,-0.0277,-0.0277,-0.0277,-0.0277,-0.0277,-0.0277,...,-0.0277,-0.0277,-0.0277,-0.0277,-0.0277,-0.0277,-0.0277,-0.0277,-0.0277,-0.0277
TCGA-FS-A4FD-10B-01D-A25P-01.bed,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,...,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008
TCGA-ZF-A9R5-10A-01D-A42G-01.bed,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,...,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066
TCGA-D3-A51J-06A-11D-A25P-01.bed,0.0408,0.0408,0.0408,0.0408,0.0408,0.0408,0.0408,0.0408,0.0408,0.0408,...,0.0408,0.0408,0.0408,0.0408,0.0408,0.0408,0.0408,0.0408,0.0408,0.0408
TCGA-KN-8433-01A-11D-2308-01.bed,-0.2363,-0.2363,-0.2363,-0.2363,-0.2363,-0.2363,-0.2363,-0.2363,-0.2363,-0.2363,...,-0.2363,-0.2363,-0.2363,-0.2363,-0.2363,-0.2363,-0.2363,-0.2363,-0.2363,-0.2363


In [21]:
len(files)

22632

In [22]:
len(s)/len(files)

57.99712796041004

In [23]:
len(d)

2584639

In [24]:
len(list(filter(lambda x: x[1]>1000, d.items())))

24

In [25]:
len(d_new)

1312591

In [26]:
sl = sorted(d_new.items())
sl[:10]

[(('chr1', 3218610), 22632),
 (('chr1', 3220101), -1),
 (('chr1', 3220160), 1),
 (('chr1', 3221051), -1),
 (('chr1', 3221621), -2),
 (('chr1', 3222113), 3),
 (('chr1', 3222334), -2),
 (('chr1', 3222470), -3),
 (('chr1', 3222627), 5),
 (('chr1', 3224658), -1)]

In [ ]:
new_list = []

chroms = sorted(set(map(lambda x:x[0],d_new.keys())))
for chrom in chroms:
    sl_chr = sorted(map(lambda x: (x[0][1],x[1]) ,filter(lambda x: x[0][0] == chrom, d_new.items())))
    pre, count = sl_chr[0]
    for curr, delta in sl_chr[1:]:
        if delta != 0:
            new_list.append((chrom, pre, curr, count))
            
        count += delta
        pre = curr


In [ ]:
len(new_list)

In [ ]:
new_list[:10]

In [ ]:
len(list(filter(lambda x:x[3] == 22632, new_list)))

In [ ]:
list(map(lambda x:x[3], new_list)))

In [ ]:
new_new_list = []
th = 0
total = 22632
c = 0
pre_chr, pre_start, pre_stop, pre_count = new_list[0]
for curr_chr, curr_start, curr_stop, curr_count in new_list:
    if curr_count + th < total:
        c+=1
        
        

c

In [ ]:
pre_stop = 0
cut = []
for curr_chr, curr_start, curr_stop, curr_count in new_list:
#     print("NEWLINE:", (curr_chr, curr_start, curr_stop, curr_count))
    if curr_start > pre_stop:
#         print("CUT:", pre_stop, curr_start, curr_start - pre_stop)
        cut.append((pre_stop, curr_start, curr_start - pre_stop))
        pre_start = curr_start
        
    pre_stop = curr_stop
    
        

In [ ]:
len(cut)

In [ ]:
cut[:10]

In [107]:
pd.read_pickle("../data/cna/tcga_cna_raw_all_samples_all_chr.pkl").head(5)

,chr1_0:4371,chr1_4372:4373,chr1_4374:9615,chr1_9616:9617,chr1_9618:12627,chr1_12628:12629,chr1_12630:18074,chr1_18075:18076,chr1_18077:18633,chr1_18634:18635,...,chr20_0:8345,chr20_8346:14270,chr20_14271:18077,chr20_18078:18080,chr21_0:164,chr21_165:5556,chr21_5557:5559,chr21_5560:15774,chr21_15775:15776,chr22_0:13709
TCGA-AA-3693-01A-01D-0903-01.bed,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.006600,-0.0066,-0.0066,-0.0066,...,-0.760800,-0.157315,0.940500,0.9405,-0.0193,-0.019300,-0.0193,-0.019300,-0.0193,0.045700
TCGA-4P-AA8J-01A-11D-A390-01.bed,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.040900,-0.0409,-0.0409,-0.0409,...,0.038100,0.038100,0.077167,0.0424,-0.0609,-0.060900,-0.0609,-0.059522,-0.0562,-0.182834
TCGA-24-0979-01A-01D-0428-01.bed,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.522517,0.5523,0.5523,-1.5957,...,-0.100367,0.367999,0.304042,0.1481,0.4255,0.346981,0.3877,0.465632,0.4889,-0.310900
TCGA-4W-AA9T-01A-11D-A390-01.bed,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.005300,0.0053,0.0053,0.0053,...,0.027900,0.020121,0.017100,0.0171,0.0276,0.027600,0.0276,0.027600,0.0276,0.006300
TCGA-5P-A9KC-01A-11D-A42I-01.bed,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.021400,-0.0214,-0.0214,-0.0214,...,-0.006000,-0.006000,-0.006000,-0.0060,-0.5160,-0.516000,-0.5160,-0.199739,-0.0135,-0.476522


In [109]:
pd.read_pickle("../data/cna/tcga_cna_raw_all_samples_all_chr_0.9_threshold.pkl").head(5)

,chr1_0:814,chr1_815:1095,chr1_1096:1509,chr1_1510:1511,chr1_1512:2235,chr1_2236:4371,chr1_4372:4373,chr1_4374:5346,chr1_5347:5351,chr1_5352:5609,...,chr22_12263:12548,chr22_12549:12550,chr22_12551:13709,chr22_13710:13715,chr22_13716:13870,chr22_13871:14465,chr22_14466:14506,chr22_14507:14508,chr22_14509:14513,chr22_14514:14530
TCGA-AA-3693-01A-01D-0903-01.bed,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,...,0.0457,0.0457,0.0457,0.0457,0.0457,0.0457,0.0457,0.0457,0.0457,0.0457
TCGA-4P-AA8J-01A-11D-A390-01.bed,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,...,-0.2245,-0.2245,-0.2245,-0.2245,-0.2245,-0.2245,-0.2245,-0.2245,-0.2245,-0.2245
TCGA-24-0979-01A-01D-0428-01.bed,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,...,-0.3109,-0.3109,-0.3109,-0.3109,-0.3109,-0.3109,-0.3109,-0.3109,-0.3109,-0.3109
TCGA-4W-AA9T-01A-11D-A390-01.bed,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,...,0.0063,0.0063,0.0063,0.0063,0.0063,0.0063,0.0063,0.0063,0.0063,0.0063
TCGA-5P-A9KC-01A-11D-A42I-01.bed,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,...,-0.4776,-0.4776,-0.4776,-0.4776,-0.4776,-0.4776,-0.4776,-0.4776,-0.4776,-0.4776
